In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

### Reading Dataset

In [2]:
s=pd.read_csv("/kaggle/input/imdb-dataset-of-top-1000-movies-and-tv-shows/imdb_top_1000.csv")

In [3]:
s

### Checking Shape,Head,Tail,Info

In [4]:
s.shape

In [5]:
s.head()

In [6]:
s.tail()

In [7]:
s.info()

### Checking important counting of Dataset

In [8]:
s.describe().T

### Checking unique value of Dataset

In [9]:
s.apply(lambda x:x.unique())

### Checking Released_year column for replacing PG value to Date_time datatype.

In [10]:
s["Released_Year"].unique()

In [11]:
s[s["Released_Year"]=='PG']

In [12]:
s["Released_Year"]=s["Released_Year"].replace({"PG":1995})

In [13]:
s["Released_Year"]=pd.to_datetime(s["Released_Year"])

In [14]:
s.info()

### Checking missing value of dataset

In [15]:
s.isna().sum()

### Filling missing values

In [16]:
s["Certificate"].mode()[0]

In [17]:
s["Certificate"].fillna(s["Certificate"].mode()[0],inplace=True)

In [18]:
s.isna().sum()

In [19]:
s["Meta_score"].agg(["mean","median","max","min","std"])

In [20]:
s["Meta_score"].fillna(s["Meta_score"].median(),inplace=True)

In [21]:
s.isna().sum()

In [22]:
s["Gross"].mode()[0]

In [23]:
s["Gross"].fillna(s["Gross"].mode()[0],inplace=True)

In [24]:
s.isna().sum()

### Replacing (,) and Changing Datatype

In [25]:
s["Gross"]=s["Gross"].str.replace(",","")

In [26]:
s["Gross"].unique()

In [27]:
s["Gross"]=s["Gross"].astype("int")

In [28]:
s.isna().sum()

In [29]:
s.columns

### Finding the year in which series release.

In [44]:
pd.DataFrame(s.groupby(["Series_Title"])["Released_Year"].value_counts())

In [35]:
import seaborn as sns
import matplotlib.pyplot as plt

### ploting countploton on Released_year column. 

In [43]:
plt.figure(figsize=(20,15))
x=sns.countplot(s["Released_Year"])
plt.title("Released_Year")
plt.xlabel("year")
plt.ylabel("counts")
plt.xticks(rotation=90);
for i in x.containers:
    x.bar_label(i)

### ploting on Certificate and Released_year columns

In [55]:
plt.figure(figsize=(20,10))
plt.bar(s["Certificate"],s["Released_Year"])
plt.xticks(rotation=50)
plt.xlabel("Certificate")
plt.ylabel("Released_Year")
plt.title("Movies Certificated Released year")

### Replacing and changing datatype of Runtime columns 

In [59]:
s["Runtime"]=s["Runtime"].str.replace("min","")

In [60]:
s["Runtime"]=s["Runtime"].astype("int")

### Finding series runtime 

In [92]:
s.groupby(["Series_Title"])["Runtime"].value_counts()


In [94]:
s["Runtime"].agg(["max","min","mean","median"])

In [95]:
s.info()

### Applying 1 sample z-test on dataset

In [103]:
x=s["Genre"].value_counts().iloc[0:10]

In [108]:
sns.countplot(x)

In [114]:
s.corr(method="spearman")

In [117]:
sns.heatmap(s.corr(method="spearman"),annot=True)

In [124]:
s["IMDB_Rating"].unique()

In [125]:
s[s["IMDB_Rating"]>=9.]

In [127]:
import pylab
import scipy.stats as st
import statsmodels
from statsmodels.stats.weightstats import ztest as zt

In [128]:
s["IMDB_Rating"].agg(["max","min","std","mean","median"])

In [130]:
s["IMDB_Rating"].mode()

In [131]:
Range=s["IMDB_Rating"].max()-s["IMDB_Rating"].min()

In [132]:
Range

In [133]:
s["IMDB_Rating"].skew()

In [134]:
s["IMDB_Rating"].kurt()

In [136]:
st.probplot(s["IMDB_Rating"],dist="norm",plot=pylab);

In [138]:
st.probplot(s["Gross"],dist="norm",plot=pylab);

In [141]:
x=st.yeojohnson(s["Gross"])[0]

In [143]:
st.probplot(x,dist="norm",plot=pylab);

In [146]:
s.info()

In [147]:
s["IMDB_Rating"]=s["IMDB_Rating"].astype("int")

In [155]:
y=np.exp(s["IMDB_Rating"])

In [157]:
st.probplot(y,dist="norm",plot=pylab);

In [172]:
s.head(5)

In [159]:
from sklearn.preprocessing import MinMaxScaler

In [160]:
scaling=MinMaxScaler()

In [183]:
scaling.fit_transform(s[["Gross"]]);

In [179]:
s["Gross"].agg(["mean","median","std","var"])

In [180]:
s["Gross"].mode()

In [189]:
st.probplot(x,dist="norm",plot=pylab);

In [205]:
x.mean()

In [191]:
x.std()

In [203]:
x1=pd.Series(x)

In [206]:
x1.iloc[0:90].mean()

In [211]:
n=90

In [213]:
z_val,p_val=zt(x1,value=44.122,alternative="two-sided")

In [214]:
print(z_val)
print(p_val)

In [215]:
alpha=0.05
if p_val>alpha:
    print("accept null hypothesis")
else:
    print("reject null hypothesis")